## Recognizing Locations from extracted text

Different methods to get the location the RTS broadcasts are about

In [2]:
from rts.db.dao import DataAccessObject
from sqlalchemy.sql import text
import pandas as pd

In [291]:
query = text("""SELECT * FROM feature;""")
df = pd.DataFrame(DataAccessObject().fetch_all(query))

In [293]:
loc_entities = []

for i, row in df.iterrows():
    try:
        for ent in row['data']['entities']:
            if ent[1] == 'LOC':
                loc_entities.append(ent[0].lower())
                # print(ent['text'])
        # print(row['data']['entities'])
    except KeyError:
        pass

In [294]:
series = pd.Series(loc_entities).value_counts()

In [295]:
all_locs = []
for s in series.items():
    all_locs.append(s)
print(all_locs)

[('suisse', 237), ('genève', 77), ('lausanne', 72), ('zurich', 51), ('est', 48), ('paris', 39), ('état', 32), ('etats-unis', 32), ('europe', 31), ('calcutta', 30), ('valais', 28), ('inde', 28), ('france', 26), ('suisses', 23), ("l'europe", 22), ('neuchâtel', 22), ('fribourg', 20), ('gagliardi', 19), ('berne', 19), ('confédération', 19), ('new york', 16), ('italie', 16), ('genevois', 15), ('syrie', 15), ('français', 14), ('allemagne', 14), ('américains', 14), ('croate', 13), ('paris saint-germain', 13), ('jura', 12), ('pékin', 12), ('nicaragua', 12), ('bâle', 11), ('los angeles', 11), ('australie', 11), ('lugano', 10), ('canada', 10), ('etat', 10), ('la france', 10), ('lausannois', 10), ('barcelone', 10), ('alpes', 9), ('suisse romande', 9), ('angleterre', 9), ('israël', 9), ('chamonix', 9), ('états-unis', 9), ('turin', 8), ('washington', 8), ('suède', 8), ('londres', 8), ('bienne', 8), ('cagliari', 8), ('lhc', 8), ('tunnel du mont-blanc', 7), ('zurichois', 7), ('canton de vaud', 7), ('

In [284]:
streets = []
for i, row in df.iterrows():
    try:
        for t in row['data']['transcript']:
            streets.append(t['t'])
        # row['data']['transcript'][0]['t']
    except KeyError:
        pass
    except TypeError:
        pass

In [240]:
for i, row in df.iterrows():
    try:
        for t in row['data']['transcript']:
            if 'rue de' in t['t'].lower():
                print(t['t'])
        # row['data']['transcript'][0]['t']
    except KeyError:
        pass
    except TypeError:
        pass

Léon Léautier attaque au tranché. Amédée Powell se fait exploser devant l'église de la Madeleine. Emile Henry fait plusieurs morts en posant des bombes, dont une au restaurant de la gare Saint-Lazare. On dynamite Faubourg Saint-Jacques, rue Saint-Martin, rue de Vaugirard, à Lyon, Angers, Loivre, Amiens, Marseille.
Et c'est ça aussi une des craintes du gouvernement, éviter que des lieux n'émergent trop spontanément. Leur concentration implique parfois des conflits de voisinage comme ce fut le cas à la rue de l'école de médecine.
Les 2 hommes sont soupçonnés d'avoir trompé 10 locataires ces dernières années. Des locataires qui auraient payé en tout plus de 200 000 francs de loyant trop sans le savoir. La plupart habitent Morges, dans cette rue derrière moi. On se rend chez un des locataires lésés. Il y a 6 mois, Alain découvrait qu'il payait 700 francs de plus par mois que l'ancien locataire, sans raison. Aujourd'hui, il accuse encore le coup.
Et là, vous étiez déjà déterminé, je vais m'

In [265]:
with open("data/geneva_streets.txt", "r") as f:
    streets = [x.strip() for x in f.readlines()]

In [287]:
def replace_street(sent):
    corrected_street = sent.replace("ruedes", "rue des")
    corrected_street = corrected_street.replace("cheminde", "chemin de")
    corrected_street = corrected_street.replace("placedes", "place des")
    corrected_street = corrected_street.replace("placede", "place de")
    corrected_street = corrected_street.replace("routedes", "route des")
    corrected_street = corrected_street.replace("avenuede", "avenue de")
    corrected_street = corrected_street.replace("avenuedu", "avenue du")
    corrected_street = corrected_street.replace("ruede", "rue de")
    corrected_street = corrected_street.replace("quaidu", "quai du")
    corrected_street = corrected_street.replace("placedu", "place du")
    corrected_street = corrected_street.replace("promenadedu", "promenade du")
    corrected_street = corrected_street.replace("ruedu", "rue du")
    corrected_street = corrected_street.replace("routede", "route de")
    corrected_street = corrected_street.replace("passagedes", "passage des")
    corrected_street = corrected_street.replace("chemindes", "chemin des")
    corrected_street = corrected_street.replace("ruedes", "rue des")
    corrected_street = corrected_street.replace("squaredu", "square du")
    corrected_street = corrected_street.replace("passagede", "passage de")
    corrected_street = corrected_street.replace("promenade des", "promenade des ")
    return sent

In [286]:
import spacy
from spacy.matcher import Matcher

# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Initialize the Matcher
matcher = Matcher(nlp.vocab)

street_names = ["Rue", "Chemin", "Place", "Avenue", "Boulevard", "Quai", "Promenade", "Route", "Square"]
connectors = ["des", "de", "du", "la", "le", "les", "l'", "d'", "au", "aux"]
second_connectors = ["l'", "d'", "la"]

# Define the pattern
pattern = [
    {"TEXT": {"IN": street_names}},
    {"TEXT": {"IN": connectors}, "OP": "?"},
    {"TEXT": {"REGEX": "^[a-zA-Z'-]+$"}, "OP": "+"},
]

# Add the pattern to the matcher
matcher.add("ADDRESS", [pattern])

# create a second pattern to match things like Rue de l'Hôtel-de-Ville or Chemin de la Gravière
pattern2 = [
    {"TEXT": {"IN": street_names}},
    {"TEXT": {"IN": connectors}, "OP": "?"},
    {"TEXT": {"IN": second_connectors}, "OP": "?"},
    {"TEXT": {"REGEX": "^[a-zA-Z-ôèéê]+$"}, "OP": "+"},

]
matcher.add("ADDRESS2", [pattern2])

# Process the sentences and get the matches
matched_streets = []

for street in streets:
    street = replace_street(street)
    # sentence = f"Hier, un accident s'est produit dans le sud de Genève, {street}, et 5 personnes ont été blessées. Le trafic est perturbé dans le secteur."
    sentence = street
    doc = nlp(sentence)
    matches = matcher(doc)

    # if matches are overlapped, we only keep the longest one
    if len(matches) > 1:
        matches = sorted(matches, key=lambda x: x[2]-x[1], reverse=True)
        matches = [matches[0]]

    for match_id, start, end in matches:
        span = doc[start:end]
        matched_streets.append(span.text)

matched_streets


['Place de la République', 'Boulevard Que']

In [279]:
doc = nlp(sentence)
matches = matcher(doc)

In [288]:
import json

In [289]:
with open("../rts/features/cities.json", "r") as f:
    cities = json.load(f)

In [298]:
city_name = series.index

In [332]:
found_clips = []
for key in cities:
    if key.lower() in city_name:
        # print(key, series[series.index == key.lower()].values[0])
        found_clips.append({
            "city": key, 
            "no_occurrences": series[series.index == key.lower()].values[0], 
            "lon": cities[key][0], "lat": cities[key][1]
        })

In [336]:
pd.DataFrame(found_clips).sort_values(by="no_occurrences", ascending=False).reset_index()[['city', 'no_occurrences', 'lon', 'lat']].to_csv("data/cities.csv", index=False)

In [338]:
cities_df = pd.DataFrame(found_clips).sort_values(by="no_occurrences", ascending=False).reset_index()[['city', 'no_occurrences', 'lon', 'lat']]

In [343]:

import folium
import numpy as np

# Create a base map centered around Switzerland
m = folium.Map(location=[46.8182, 8.2275], zoom_start=8)

# Add city points to the map
for index, row in cities_df.iterrows():
    folium.CircleMarker(
        location=(row['lon'], row['lat']),
        radius=np.sqrt(row['no_occurrences'] / np.pi) * 5,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['city'] + ': ' + str(row['no_occurrences']) + ' occurrences'
    ).add_to(m)

m
